In [ ]:
# NOTES FROM MEETING
# No cell should take more than 20 minutes to run
# Please limit to one temporary collection (must be deleted at end of notebook). No external files
# Try to reduce single-line cells

# Mini-competition -
# Challenge: Limit to 75 lines of code (100 for team 1)! (excluding imports)
# Fit Success Rate: Fitting method with highest fit success rate
# Fit Average error: Fitting method with lowest fit error rate
# Fit Time complexity: Fitting method with lowest time expended
# Notebook housekeeping: Cleanest-looking notebook win!

# Sigmoid Criteria Curve Fitting: CDF Approach
**Contributors:** 

**Achievement:**

**Requirements:**


## Example of cell markdown
**This is a note:** Please have markdown before each cell. In this format!

In [ ]:
# Import packages you need
import sys
sys.path.append('../..')
from pymongo import UpdateOne
from pymongo import MongoClient
from tqdm import tqdm
from exploration.config import mongo_inst
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm 
import scipy
import seaborn as sns
from scipy.optimize import curve_fit
from mlpp.data_modeling.sigmoid_fitting import fit_normal_cdf

In [ ]:
# Instantiate your mongo instance, classes, and any config/global vars
client = MongoClient(port=27017)
db = client.osu_random_db
bm_ids = list(map(lambda c: c['_id'], db['beatmap_criteria_curve'].find({}, {})))

In [ ]:
bm_data = {}
for i in bm_ids:
    bm_data[i] = list(db['beatmap_criteria_curve'].find({"_id": i}, {"_id": 0, "no_mod.total": 1, "no_mod.n_pass": 1}))

In [ ]:
# find a good user
first_bm_total = bm_data[104229][0]["no_mod"]["total"]
first_bm_n_pass = bm_data[104229][0]["no_mod"]["n_pass"]

In [ ]:
ind_good = []
k = 0
for total in first_bm_total: 
    if (total != 0):
        ind_good.append(k)
    k = k + 1

In [ ]:
first_bm_good_total = [first_bm_total[i] for i in ind_good]
first_bm_good_npass = [first_bm_n_pass[i] for i in ind_good]

In [ ]:
x = range(len(first_bm_good_total))
y = np.asarray(first_bm_good_npass) / np.asarray(first_bm_good_total)

In [ ]:
def helper_take_out_zero(bm_total):
    good_index = []
    k = 0
    for total in bm_total: 
        if (total != 0):
            good_index.append(k)
        k = k + 1
    bm_good_total = [bm_total[i] for i in good_index]
    return bm_good_total
len(helper_take_out_zero(first_bm_total))



In [ ]:
n = len(first_bm_good_total)
x = range(0, n) 
f = lambda x, mu, sigma: scipy.stats.norm(mu,sigma).cdf(x)

mu, sigma = scipy.optimize.curve_fit(f, x, y)[0]

# plt.scatter(x, y)

total = 0
for i in x:
    error = (y[i] - scipy.stats.norm(mu,sigma).cdf(i))**2
    total += error
print(total / n)
    

# scipy.stats.norm(mu,sigma).cdf(x)
# scipy.optimize.curve_fit(f,x,y)

In [ ]:
fit_normal_cdf(first_bm_total, y)

In [ ]:
# Load the curve data

In [ ]:
# Run all CDF curve fits & store in beatmap_criteria_curve collection

In [ ]:
# Load the beatmap attribs data

In [ ]:
# Visualisations of correlation between CDF mean/std with attribs

In [ ]:
# fit success-rate summary table with (3 rows 1 for each CDF fit) (4 columns 1* - 4*, 4* - 6*, 6* plus, all)

In [ ]:
# for curves that could be fit, error summary table with same layout as above

## Conclusion
Don't forget to provide a conclusion